In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, func
#from sqlalchemy.ext.automap import automap_base
#from sqlalchemy.orm import Session

In [2]:
#nrows=10
Grocery_json = pd.read_json('Resources/Grocery_and_Gourmet_Food.json',lines=True)


In [3]:
Grocery_json

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
0,5,True,"06 4, 2013",ALP49FBWT4I7V,1888861614,Lori,Very pleased with my purchase. Looks exactly l...,Love it,1370304000,NaN,NaN,NaN
1,4,True,"05 23, 2014",A1KPIZOCLB9FZ8,1888861614,BK Shopper,Very nicely crafted but too small. Am going to...,Nice but small,1400803200,NaN,NaN,NaN
2,4,True,"05 9, 2014",A2W0FA06IYAYQE,1888861614,daninethequeen,still very pretty and well made...i am super p...,"the ""s"" looks like a 5, kina",1399593600,NaN,NaN,NaN
3,5,True,"04 20, 2014",A2PTZTCH2QUYBC,1888861614,Tammara,"I got this for our wedding cake, and it was ev...",Would recommend this to a friend!,1397952000,NaN,NaN,NaN
4,4,True,"04 16, 2014",A2VNHGJ59N4Z90,1888861614,LaQuinta Alexander,It was just what I want to put at the top of m...,Topper,1397606400,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5074155,5,True,"10 4, 2018",AGOPF2VTEOP57,B01HJHSVG6,Bob,"Love this product, very fresh, complete full f...",Exceclent product,1538611200,NaN,NaN,{'Size:': ' 2.29 oz'}
5074156,3,True,"10 3, 2018",A5X5TI4JCH9CN,B01HJHSVG6,Jacqueline Schafer,Good taste but small quantity for the expensiv...,Good taste but small quantity for the price us...,1538524800,NaN,NaN,{'Size:': ' 2.29 oz'}
5074157,4,True,"10 2, 2018",A1WMQQLC3TVAFI,B01HJHSVG6,Motionjunkie,Unlike some of the bad reviews regarding aroma...,See Comments below,1538438400,NaN,NaN,{'Size:': ' 2.29 oz'}
5074158,5,True,"09 30, 2018",ALVP6JZRTEDY6,B01HJHSVG6,Marissa,I absolutely love this vanilla bean paste. I h...,Love!!,1538265600,NaN,NaN,{'Size:': ' 2.29 oz'}


In [31]:
Grocery_json.columns


Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'image', 'style', 'reviewlength'],
      dtype='object')

In [32]:
#read labeled csv
labeled_df = pd.read_csv('Resources/labeled.csv')
labeled_df

,words,sentiment
0,"['comforting', 'know', 'processed', 'like', 's...",pos
1,"['never', 'disappointed', 'cup', 'cider', 'lik...",neg
2,"['never', 'ever', 'write', 'reviews', 'product...",neg
3,"['tasty', 'package', 'arrived', 'neatly', 'pac...",pos
4,"['gonna', 'buy', 'future', 'ate', 'completely'...",pos
...,...,...
809635,"['great', 'tasting', 'tuna', 'easy', 'open', '...",pos
809636,"['addicted', 'product', 'grocer', 'quit', 'car...",neg
809637,"['sure', 'happened', 'box', 'arrived', 'comple...",neg
809638,"['put', 'water', 'bottle', 'done', 'working', ...",pos


In [33]:
#count the number of words in a review
Grocery_json["reviewlength"] = labeled_df["words"].apply(lambda x: len(x))

Grocery_json.head(50)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style,reviewlength
0,5,True,"06 4, 2013",ALP49FBWT4I7V,1888861614,Lori,Very pleased with my purchase. Looks exactly l...,Love it,1370304000,NaN,NaN,NaN,270.0
1,4,True,"05 23, 2014",A1KPIZOCLB9FZ8,1888861614,BK Shopper,Very nicely crafted but too small. Am going to...,Nice but small,1400803200,NaN,NaN,NaN,287.0
2,4,True,"05 9, 2014",A2W0FA06IYAYQE,1888861614,daninethequeen,still very pretty and well made...i am super p...,"the ""s"" looks like a 5, kina",1399593600,NaN,NaN,NaN,1156.0
3,5,True,"04 20, 2014",A2PTZTCH2QUYBC,1888861614,Tammara,"I got this for our wedding cake, and it was ev...",Would recommend this to a friend!,1397952000,NaN,NaN,NaN,126.0
4,4,True,"04 16, 2014",A2VNHGJ59N4Z90,1888861614,LaQuinta Alexander,It was just what I want to put at the top of m...,Topper,1397606400,NaN,NaN,NaN,92.0
5,1,True,"02 21, 2014",ATQL0XOLZNHZ4,1888861614,Brianna Small,"The S is beautiful, however upon checkout it i...",shipping disaster,1392940800,NaN,NaN,NaN,97.0
6,5,True,"12 1, 2013",A94ZG5CWN70E7,1888861614,trinette claude,Omg.. The S was inexpensive and EXACTLY what w...,Great quality for this product. I have never b...,1385856000,NaN,NaN,NaN,22.0
7,3,True,"11 6, 2013",A3QBT8YC3CZ7C9,1888861614,Rela1982,It was a nice size for a cake topper and it wa...,Great Buy.,1383696000,NaN,NaN,NaN,96.0
8,5,True,"10 2, 2013",AGKW3A1RB1YGO,1888861614,"Marilyn Hatboro,Pa",Perfect!!! Can not wait to use on my Wedding c...,Very Pretty!!,1380672000,NaN,NaN,NaN,57.0
9,5,True,"09 6, 2013",A1B65IWLUVOUQB,1888861614,KASmith,This was exactly what we were looking for to m...,Perfect!,1378425600,NaN,NaN,NaN,44.0


In [34]:
groceries_df = Grocery_json[['asin','overall','reviewerID', 'reviewTime','unixReviewTime', 'reviewText', "reviewlength"]]
groceries_df

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
1,1888861614,4,A1KPIZOCLB9FZ8,"05 23, 2014",1400803200,Very nicely crafted but too small. Am going to...,287.0
2,1888861614,4,A2W0FA06IYAYQE,"05 9, 2014",1399593600,still very pretty and well made...i am super p...,1156.0
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126.0
4,1888861614,4,A2VNHGJ59N4Z90,"04 16, 2014",1397606400,It was just what I want to put at the top of m...,92.0
...,...,...,...,...,...,...,...
5074155,B01HJHSVG6,5,AGOPF2VTEOP57,"10 4, 2018",1538611200,"Love this product, very fresh, complete full f...",NaN
5074156,B01HJHSVG6,3,A5X5TI4JCH9CN,"10 3, 2018",1538524800,Good taste but small quantity for the expensiv...,NaN
5074157,B01HJHSVG6,4,A1WMQQLC3TVAFI,"10 2, 2018",1538438400,Unlike some of the bad reviews regarding aroma...,NaN
5074158,B01HJHSVG6,5,ALVP6JZRTEDY6,"09 30, 2018",1538265600,I absolutely love this vanilla bean paste. I h...,NaN


In [35]:
#drop dupicates
groceries_df = groceries_df.drop_duplicates()
groceries_df

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
1,1888861614,4,A1KPIZOCLB9FZ8,"05 23, 2014",1400803200,Very nicely crafted but too small. Am going to...,287.0
2,1888861614,4,A2W0FA06IYAYQE,"05 9, 2014",1399593600,still very pretty and well made...i am super p...,1156.0
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126.0
4,1888861614,4,A2VNHGJ59N4Z90,"04 16, 2014",1397606400,It was just what I want to put at the top of m...,92.0
...,...,...,...,...,...,...,...
5074155,B01HJHSVG6,5,AGOPF2VTEOP57,"10 4, 2018",1538611200,"Love this product, very fresh, complete full f...",NaN
5074156,B01HJHSVG6,3,A5X5TI4JCH9CN,"10 3, 2018",1538524800,Good taste but small quantity for the expensiv...,NaN
5074157,B01HJHSVG6,4,A1WMQQLC3TVAFI,"10 2, 2018",1538438400,Unlike some of the bad reviews regarding aroma...,NaN
5074158,B01HJHSVG6,5,ALVP6JZRTEDY6,"09 30, 2018",1538265600,I absolutely love this vanilla bean paste. I h...,NaN


In [36]:
rating_5 = groceries_df.loc[groceries_df['overall'] == 5]
rating_5

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126.0
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22.0
8,1888861614,5,AGKW3A1RB1YGO,"10 2, 2013",1380672000,Perfect!!! Can not wait to use on my Wedding c...,57.0
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44.0
...,...,...,...,...,...,...,...
5074153,B01HJHSVG6,5,A1VL40O7D88AY0,"10 4, 2018",1538611200,Love this Vanilla. Blends and bakes beautifully.,NaN
5074154,B01HJHSVG6,5,A1LBAF0CNISB04,"10 4, 2018",1538611200,You'll never go back to plain ol' Vanilla Extr...,NaN
5074155,B01HJHSVG6,5,AGOPF2VTEOP57,"10 4, 2018",1538611200,"Love this product, very fresh, complete full f...",NaN
5074158,B01HJHSVG6,5,ALVP6JZRTEDY6,"09 30, 2018",1538265600,I absolutely love this vanilla bean paste. I h...,NaN


In [37]:
rating_1 = groceries_df.loc[groceries_df['overall'] == 1]
rating_1

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
5,1888861614,1,ATQL0XOLZNHZ4,"02 21, 2014",1392940800,"The S is beautiful, however upon checkout it i...",97.0
22,4639725183,1,AE937MVR5J8OL,"12 10, 2016",1481328000,Tasted old and not individually bag,104.0
27,4639725183,1,A27LF8222OIRRN,"09 10, 2016",1473465600,Not the great taste everyone was talking about :(,40.0
37,4639725183,1,A34RH95REHOQFO,"09 25, 2015",1443139200,Terrible. What a waste of money. This is not t...,1391.0
69,4639725043,1,A3ELDSSPP9RZPG,"03 31, 2017",1490918400,The tea is so fine it leaves tea particles flo...,194.0
...,...,...,...,...,...,...,...
5074110,B01HJ7QPKU,1,A1ZO0PE7RT0IKK,"05 19, 2017",1495152000,Followed instructions on the package to prepar...,NaN
5074111,B01HJ4R980,1,A1T8D0SC1P04CZ,"09 8, 2018",1536364800,For my grandson. Arrived broken into crumbs.,NaN
5074112,B01HJ4R980,1,A3TG44X2A5WTP5,"08 27, 2018",1535328000,These are delicious cookies - just don't order...,NaN
5074120,B01HJ4R980,1,A33SA5P6IOEATJ,"10 10, 2016",1476057600,Can't say he product is good as is practically...,NaN


In [38]:
rating_2 = groceries_df.loc[groceries_df['overall'] == 2]
rating_2

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
46,4639725183,2,A1EDAUU1B7YGTD,"05 7, 2015",1430956800,Mediocre taste and not like the yellow label f...,128.0
55,4639725183,2,A36N7FI7QREIH7,"01 10, 2015",1420848000,"Lipton Yellow is normally great premium tea, b...",125.0
77,4639725043,2,AA3RFH4XSLRF,"01 28, 2017",1485561600,"Good for Lipton fans maybe, but not so good fo...",148.0
100,4639725043,2,A3T7ACW6DIECL,"09 8, 2015",1441670400,Just not that good.,896.0
166,5463213682,2,A38YPICAU2VDU8,"09 10, 2016",1473465600,i love reishi coffee...but I prefer no sugar. ...,37.0
...,...,...,...,...,...,...,...
5074029,B01HIWL8UI,2,A38ME72ZVZ3UQD,"09 15, 2016",1473897600,"Didn't care for the pork, the rest were just OK.",NaN
5074084,B01HJ4FWGG,2,A1CHF9VHRDTTO5,"09 10, 2018",1536537600,"I received this candy today, it arrived in pr...",NaN
5074115,B01HJ4R980,2,AHDO6UIU1Y68O,"10 12, 2017",1507766400,Cookies came broken in pieces.,NaN
5074116,B01HJ4R980,2,A2RZH38UWWX059,"09 25, 2017",1506297600,These were smashed to bits upon delivery. For ...,NaN


In [39]:
rating_3 = groceries_df.loc[groceries_df['overall'] == 3]
rating_3

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
7,1888861614,3,A3QBT8YC3CZ7C9,"11 6, 2013",1383696000,It was a nice size for a cake topper and it wa...,96.0
40,4639725183,3,A165AME4SLYDWD,"08 23, 2015",1440288000,Same product and name as the tea I've bought a...,29.0
60,4639725183,3,A29LPNLNZ7E2XI,"11 23, 2014",1416700800,"Normally this tea rates 5 stars, it has a wond...",104.0
67,4639725043,3,A1YA5QHMETOVC8,"05 21, 2009",1242864000,I have been a Lipton loose tea junkie for 50 y...,20.0
68,4639725043,3,ACOICLIJQYECU,"04 11, 2008",1207872000,I have recently started drinking hot tea again...,189.0
...,...,...,...,...,...,...,...
5074089,B01HJ2VP9G,3,A2OWJLIXWRH7SI,"02 1, 2018",1517443200,"I'm a spice fiend, but I can't say this add al...",NaN
5074134,B01HJFC704,3,A143ZDYT3UZSJR,"08 1, 2018",1533081600,"I expected more truffle flavor, especially for...",NaN
5074144,B01HJF6FRA,3,A1P0X9E6F99J4T,"05 20, 2017",1495238400,Needs improvement to make it taste like real r...,NaN
5074149,B01HJEK5QS,3,A3NY9RMNIODSDW,"09 12, 2016",1473638400,Expensive and could have bought at local store...,NaN


In [40]:
rating_4 = groceries_df.loc[groceries_df['overall'] == 4]
rating_4

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
1,1888861614,4,A1KPIZOCLB9FZ8,"05 23, 2014",1400803200,Very nicely crafted but too small. Am going to...,287.0
2,1888861614,4,A2W0FA06IYAYQE,"05 9, 2014",1399593600,still very pretty and well made...i am super p...,1156.0
4,1888861614,4,A2VNHGJ59N4Z90,"04 16, 2014",1397606400,It was just what I want to put at the top of m...,92.0
17,4639725183,4,A21ZU2TINEQE0H,"02 8, 2014",1391817600,Tried this while overseas last year; don't rem...,61.0
38,4639725183,4,A19TFNGQV7GPSX,"09 21, 2015",1442793600,"Just so everyone knows, this IS the orange pek...",27.0
...,...,...,...,...,...,...,...
5074133,B01HJ9Y7OO,4,A2CC652MJ0FX7E,"07 27, 2016",1469577600,This item arrived a little faster than the ori...,NaN
5074135,B01HJFC704,4,AO0QVVFS0P2GU,"08 15, 2017",1502755200,OK and without eggs.,NaN
5074146,B01HJFBN9A,4,A1AZD3IIOFDLPC,"03 12, 2018",1520812800,Very tasty. The individual packaging creates a...,NaN
5074151,B01HJHSVG6,4,A1EET7MJO4KKX2,"10 7, 2018",1538870400,This is only the 2nd kind of vanilla paste I h...,NaN


In [41]:
groceries_df_12_45 = groceries_df[groceries_df.overall != 3]
groceries_df_12_45

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
1,1888861614,4,A1KPIZOCLB9FZ8,"05 23, 2014",1400803200,Very nicely crafted but too small. Am going to...,287.0
2,1888861614,4,A2W0FA06IYAYQE,"05 9, 2014",1399593600,still very pretty and well made...i am super p...,1156.0
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126.0
4,1888861614,4,A2VNHGJ59N4Z90,"04 16, 2014",1397606400,It was just what I want to put at the top of m...,92.0
...,...,...,...,...,...,...,...
5074154,B01HJHSVG6,5,A1LBAF0CNISB04,"10 4, 2018",1538611200,You'll never go back to plain ol' Vanilla Extr...,NaN
5074155,B01HJHSVG6,5,AGOPF2VTEOP57,"10 4, 2018",1538611200,"Love this product, very fresh, complete full f...",NaN
5074157,B01HJHSVG6,4,A1WMQQLC3TVAFI,"10 2, 2018",1538438400,Unlike some of the bad reviews regarding aroma...,NaN
5074158,B01HJHSVG6,5,ALVP6JZRTEDY6,"09 30, 2018",1538265600,I absolutely love this vanilla bean paste. I h...,NaN


In [42]:
#choosing every second row of the five star rating for a total of 600,000 to reduce its size from around 3.5 million ratings to a number closer to the 1 star ratings. 
rating_5_trimmed = rating_5[:1200000:2]
rating_5_trimmed

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22.0
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44.0
11,4639725183,5,A1QVBUH9E1V6I8,"11 19, 2014",1416355200,No adverse comment.,257.0
13,4639725183,5,A2CHH5U12THP2D,"11 3, 2014",1414972800,Best black tea in US.\n\nHighly recommend.\nI ...,33.0
...,...,...,...,...,...,...,...
1659658,B005258A0K,5,AJQN9SH9FSPB1,"06 10, 2015",1433894400,Amazing breath freshener.\nDoes stick a little...,NaN
1659662,B005258A0K,5,AV37026DN7QFF,"06 6, 2015",1433548800,My daughter has started chewing gum to help ke...,NaN
1659667,B005258A0K,5,A2ID6N65Y5QYDT,"05 15, 2015",1431648000,This is the best xylitol gum on the market! L...,NaN
1659670,B005258A0K,5,A3V1S70XDFIPA,"05 7, 2015",1430956800,Nice to be able to get gum without aspartame i...,NaN


In [43]:
groceries_df_1_5 = [rating_5_trimmed, rating_1]
groceries_df_1_5
groc_1_5 = pd.concat(groceries_df_1_5)
groc_1_5

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22.0
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44.0
11,4639725183,5,A1QVBUH9E1V6I8,"11 19, 2014",1416355200,No adverse comment.,257.0
13,4639725183,5,A2CHH5U12THP2D,"11 3, 2014",1414972800,Best black tea in US.\n\nHighly recommend.\nI ...,33.0
...,...,...,...,...,...,...,...
5074110,B01HJ7QPKU,1,A1ZO0PE7RT0IKK,"05 19, 2017",1495152000,Followed instructions on the package to prepar...,NaN
5074111,B01HJ4R980,1,A1T8D0SC1P04CZ,"09 8, 2018",1536364800,For my grandson. Arrived broken into crumbs.,NaN
5074112,B01HJ4R980,1,A3TG44X2A5WTP5,"08 27, 2018",1535328000,These are delicious cookies - just don't order...,NaN
5074120,B01HJ4R980,1,A33SA5P6IOEATJ,"10 10, 2016",1476057600,Can't say he product is good as is practically...,NaN


In [44]:
g1_5 = pd.merge(rating_5_trimmed, rating_1, how='outer')
g1_5

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
1,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22.0
2,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44.0
3,4639725183,5,A1QVBUH9E1V6I8,"11 19, 2014",1416355200,No adverse comment.,257.0
4,4639725183,5,A2CHH5U12THP2D,"11 3, 2014",1414972800,Best black tea in US.\n\nHighly recommend.\nI ...,33.0
...,...,...,...,...,...,...,...
1005079,B01HJ7QPKU,1,A1ZO0PE7RT0IKK,"05 19, 2017",1495152000,Followed instructions on the package to prepar...,NaN
1005080,B01HJ4R980,1,A1T8D0SC1P04CZ,"09 8, 2018",1536364800,For my grandson. Arrived broken into crumbs.,NaN
1005081,B01HJ4R980,1,A3TG44X2A5WTP5,"08 27, 2018",1535328000,These are delicious cookies - just don't order...,NaN
1005082,B01HJ4R980,1,A33SA5P6IOEATJ,"10 10, 2016",1476057600,Can't say he product is good as is practically...,NaN


In [45]:
#groceries_meta = pd.read_csv('Resources/fakemeat_meta.csv')
#groceries_meta

In [46]:
#groceries_meta = groceries_meta['price'].dropna()
#groceries_meta

In [47]:
#groceries_meta.count()

In [48]:
#groceries15merged = pd.merge(g1_5, groceries_meta, on='asin', how='outer')
#groceries15merged = [g1_5, groceries_meta]
#groceries15merged


In [49]:
#groc_1_5 = pd.concat(groceries15merged)
#groc_1_5

In [50]:
g1_5 = groc_1_5.dropna()
g1_5

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270.0
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22.0
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44.0
11,4639725183,5,A1QVBUH9E1V6I8,"11 19, 2014",1416355200,No adverse comment.,257.0
13,4639725183,5,A2CHH5U12THP2D,"11 3, 2014",1414972800,Best black tea in US.\n\nHighly recommend.\nI ...,33.0
...,...,...,...,...,...,...,...
809605,B0012A8GVC,1,A2A6ITU78093LW,"03 3, 2016",1456963200,not good.,27.0
809614,B0012A8GVC,1,A29J9H9L91M7YU,"12 9, 2015",1449619200,"When I received 2 bags of this icing, the bag ...",42.0
809627,B0012A8GVC,1,A326774B3J1SGE,"08 29, 2015",1440806400,Ordered this by mistake thinking it was mering...,47.0
809636,B0012A8GVC,1,A1KBC4IUIIDYDV,"03 25, 2015",1427241600,I ordered 2 bags. One bag was completely slit ...,246.0


In [51]:
groceries_csv = g1_5.to_csv(path_or_buf='Resources/groceries_merged_trimmed.csv', index=False)


In [53]:
#unique_values = g1_5.brand.unique()
#unique_values

In [54]:
#only fake meat products: ratings, words in review, date, price
#groc_1_5_fakemeat = groc_1_5[(groc_1_5['brand']== 'Tofurky') & (groc_1_5['brand']=='Boca') & (groc_1_5['brand']=='Quorn') & (groc_1_5['brand']== 'Field Roast') & (groc_1_5['brand']=='Gardein') &
      # (groc_1_5['brand']=='Simply Balanced') & (groc_1_5['brand']=='Beyond Meat') & (groc_1_5['brand']=='Sweet Earth Natural Foods')]
#groc_1_5_fakemeat 

In [55]:
# only fakemeat: ratings, words in review, date, price
#fakemeat_df = groc_1_5_fakemeat[['brand','overall','reviewTime','unixReviewTime', 'reviewText', "reviewlength", 'price']]
#fakemeat_df

In [56]:
# sqlalchemy's create engine preparing the ability to connect to a sqlite database. then with .connect connecting to the database. 
engine = create_engine('sqlite:///groc15trimmed.sqlite', echo=True)
sqlite_connection = engine.connect()

In [57]:
#connecting to database and inputting info in from dataframe
sqlite_table = "groceriesALL"
ALLgroceries=g1_5.to_sql(sqlite_table, sqlite_connection, if_exists='replace')

2021-06-21 12:20:46,057 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("groceriesALL")
2021-06-21 12:20:46,058 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-21 12:20:46,060 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("groceriesALL")
2021-06-21 12:20:46,060 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-21 12:20:46,062 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-06-21 12:20:46,063 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-21 12:20:46,064 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("groceriesALL")
2021-06-21 12:20:46,065 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-21 12:20:46,068 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-06-21 12:20:46,069 INFO sqlalchemy.engine.Engine [raw sql] ('groceriesALL',)
2021-06-21 12:20:46,070 INFO sqlalchemy.engine.Engine PRAGMA main.foreig

In [58]:
#select and show data in groceries ALL table
engine.execute('SELECT * FROM groceriesALL').fetchall()

2021-06-21 12:20:53,862 INFO sqlalchemy.engine.Engine SELECT * FROM groceriesALL
2021-06-21 12:20:53,863 INFO sqlalchemy.engine.Engine [raw sql] ()


[(0, '1888861614', 5, 'ALP49FBWT4I7V', '06 4, 2013', 1370304000, 'Very pleased with my purchase. Looks exactly like the picture and will look great on my cake. It definitely will sparkle.', 270.0),
 (6, '1888861614', 5, 'A94ZG5CWN70E7', '12 1, 2013', 1385856000, 'Omg.. The S was inexpensive and EXACTLY what we wanted. I searched everywhere and this one was the best one.', 22.0),
 (9, '1888861614', 5, 'A1B65IWLUVOUQB', '09 6, 2013', 1378425600, 'This was exactly what we were looking for to make our cake sparkle along with the strands of diamonds we already have on the cake!', 44.0),
 (11, '4639725183', 5, 'A1QVBUH9E1V6I8', '11 19, 2014', 1416355200, 'No adverse comment.', 257.0),
 (13, '4639725183', 5, 'A2CHH5U12THP2D', '11 3, 2014', 1414972800, "Best black tea in US.\n\nHighly recommend.\nI use 3 bags in a large 16 oz glass mug with boiled water then add boiled milk & sugar. Oh my, it's wonderful. I wish I could drink it at night.", 33.0),
 (15, '4639725183', 5, 'A378HXZATS9HKM', '05 2